### General Approach for solving using OR-Tools

### Good ref: https://developers.google.com/optimization/introduction/python
- Import the required libraries,
- Declare the solver,
- Create the variables (decision variables/intermediate variables that might help with constraints etc.)
    - N.B. Uses declarative programming to define variable dtype from offset -cannot convert it to different type thereafter
- Define the constraints,
- Define the objective function,
- Using the above, should then be able to identify type of problem you are dealing with, and check appropriate solver used - amend if not
- Invoke the solver
- Display the results

In [ ]:
# status
# pywraplp.Solver.OPTIMAL = 0
# pywraplp.Solver.FEASIBLE = 1
# pywraplp.Solver.INFEASIBLE = 2
# pywraplp.Solver.UNBOUNDED = 3
# pywraplp.Solver.ABNORMAL = 4
# pywraplp.Solver.NOT_SOLVED = 6

# # which linear programming solver to use? # 
# pywraplp.Solver.CreateSolver()
#   - CLP_LINEAR_PROGRAMMING or CLP
#   - CBC_MIXED_INTEGER_PROGRAMMING or CBC
#   - GLOP_LINEAR_PROGRAMMING or GLOP
#   - BOP_INTEGER_PROGRAMMING or BOP
#   - SAT_INTEGER_PROGRAMMING or SAT or CP_SAT
#   - SCIP_MIXED_INTEGER_PROGRAMMING or SCIP
#   - GUROBI_LINEAR_PROGRAMMING or GUROBI_LP
#   - GUROBI_MIXED_INTEGER_PROGRAMMING or GUROBI or GUROBI_MIP
#   - CPLEX_LINEAR_PROGRAMMING or CPLEX_LP
#   - CPLEX_MIXED_INTEGER_PROGRAMMING or CPLEX or CPLEX_MIP
#   - XPRESS_LINEAR_PROGRAMMING or XPRESS_LP
#   - XPRESS_MIXED_INTEGER_PROGRAMMING or XPRESS or XPRESS_MIP
#   - GLPK_LINEAR_PROGRAMMING or GLPK_LP
#   - GLPK_MIXED_INTEGER_PROGRAMMING or GLPK or GLPK_MIP

In [3]:
# !pip install ortools

In [4]:
import pandas as pd
import numpy as np
import time
from ortools.sat.python import cp_model  # constraint programming
from ortools.linear_solver import pywraplp # python wrapper for several different libraries for linear and mixed-integer optimization, including third-party libraries
from ortools.sat.python import cp_model
from ortools.init import pywrapinit
import re
import random

class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__solution_list = []

    def on_solution_callback(self):
        self.__solution_count += 1
        print([self.Value(v) for v in self.__variables])
        self.__solution_list.append([self.Value(v) for v in self.__variables])
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
            print()
            # self.__solution_list += self.Value(v)
        print()
        return self.__solution_list

    def solution_count(self):
        return self.__solution_count
    
    def solution_list(self):
        return self.__solution_list
    
class VarArraySolutionCollector(cp_model.CpSolverSolutionCallback):

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.solution_list = []

    def on_solution_callback(self):
        self.solution_list.append([self.Value(v) for v in self.__variables])

# Discussion

1. What do we want the output of the BTO model to be?
    a) Deterministic BTO Value
        - Overestimates the predictive ability of the model
        - Does not appreciate customer accepting of a range of offers
    b) BTO Range based on quantile assignment
        - current preference
2. Downgrade Propensity
    - can we use the current values produced by existing models in production?
        - models predict call likelihood and do not factor offer given on the call - leakage of sorts 
        - we want DG propensity given they have called and given a specific offer amt (essentially what RCT includes)
        - how well calibrated are DGs across different product models
        - n.b. looked into individual models and cannot tweak single feature input to account for suggested offer
            - HD model has features like current tv offer amt, current contract tv offer amt, current HD offer amt, even things like days_until tv offer end which is msot important in most will have to be re-imputed for accurately calibrated propensity etc. Cannot expect to impute new values for all of these and recalculate DG propensity .....
            - https://confluence.bskyb.com/display/BusInt/Final+HD+Downgrade+Model
            - https://confluence.bskyb.com/display/BusInt/Final+Model+-+Kids+Downgrade
    - Is it feasible to create our own models for this based on RCT?
        - i.e. recalibrates DG proba in response to offer_amt suggested
        - Heavily simplified model with ~10 predictors 
        - Might at least give us a steer on how important offer_amt_other is vs offer_amt_product based on feature importance plots
3. Meet with Richard to discuss further needs/additional unconsidered constraints/get guidance? 
4. Business level constraints (i.e. part B) rather than just customer level constraints
    - Adapt part A approach
        - introduce parameter in allocation score formula??? 
        - 
    - Cohort approach, where similiar cohorts given same offers
        - How to define cohort? 
        - Cohort number could explode out very easily (esp if split by product combinations / L/M/H propensity /  ..)
5. Still need to nail down KPI (NRC/NCC etc.)


# Outstanding Questions
1. Can we better understand the offers we allocate? 
    - Are they all 18mth or are some of them monthly? 
    - If Cinema offer is monthly whereas Sports is 18 month, this will of course matter to the customer and to our NC expectation
2. Can we scope out further constraints? 
    - what if existing offers on products 
        - e.g. if £x on Sports already then cannot give £7<
    - can we apply offers where in-contract, existing offers

# Potential Approaches

- Is the total offer amount most important OR is the distribution of offers most important ?? Whichever view you take could determine method chosen
- The following all assume that we can use DG propensity as a perfectly fine way to choose between products:
1. Optimise (MIP) distance from BTO target & best offer allocation (i.e. assumes output of regression model is single deterministic value)
    - assumes total offer amount most important factor
2. Optimise (MIP) for best offer allocation s.t constraint that total offer amount must be between BTO_low and BTO_high
    - added flexibility that considers offer amount as important but appreciates BTO model estimation has limited accuracy & customer willingness to accept is non-deterministic
    - emphasis instead on spreading of offers where potential risk to specific products
    - understands some customers have strong preference to remove single products
3. CP Optimisation to get feasible subsets between BTO_low and BTO_high before externally choosing offer combo with max allocation score
    - same rationale as 2.
    - potentially just quicker?? 
    - does not require an objective function
    - might not have enough constraints to reduce this down to a small enough subset .. 

- Useful resources
    - https://xiang.es/posts/cp-sat/

## To do

- Unit tests for edge cases 
    - if no solutions because not enough products to reach BTO_lower
    - if no solutions because ....
    - if no downgrade scores

- Further constraints?
    - If they are in-contract for a product, maybe cannot give further offer i.e. capped at 0
    - If they are on a specific deep offer, maybe cannot give further offer

- Scalability 
    - As part of this better understand where the bottlenecks are when running on scale
        - e.g. building the actual solver model
        - CS thinks 400K limit before kernel disconnects
    - measure how long for 10mn TV customers with 5 product offers available & 4 levels each
        - can it do this or is there a threshold point?
    - as product number increases
        - Test for Sports,Cinema,Kids,HD,Signature
    - as number of customers (rows) increase
        - what is the ceiling for customers we need to produce optimised offers for?
            - i.e. 10MN TV Base, but only who can we remove based on 
        - how many calls do we get in a week? e.g. 50K
            - Test for 50K customers assuming that we can perfectly predict who these customers will be and return their optimal offers i.e. minimises computational effort/wasteage
        - using call propensity model, if we just optimise the top quartile of customers would this cover 99% of callers?? 
            - Maybe test how good our model is first before applying this... and figure out how many customers we'd need to optimise for that would cover 99% of them
        - how long would it take if we did in fact 
        
- How to test out our offer allocation formula
    - to help tune hyperparameters like net contribution scaling power

__Create toy data & define parameters__

In [5]:
n_repeats = 1 # duplicates of this cust set to test for scalability
cust_df = pd.DataFrame({'account_number':[123,232,321,132],
                        'BTO_low':[10,15,5,30],
                        'BTO_high':[13,17,6,40],
                        'BTO_target':[12,16,6,32], # could either be the deterministic prediction of BTO model or midpoint of quantile-based approach
                        'LS_Cap':[20,15,7,50],
                        'Sports_Active':[1,1,0,1],
                        'Cine_Active':[1,1,0,1],
                        'Kids_Active':[1,1,1,1],
                        'HD_Active':[0,0,1,0],
                        # Can just impute np.nan with 0 if do not have product #
                        'Sports_DG':[0.8,0.4,0,0.6],
                        'Cine_DG':[0.9,0.8,0,0.1],
                        'Kids_DG':[0.5,0.1,0.9,0.5],
                        'HD_DG':[0,0,0.9,0],
                        'Signature_DG':[0.3,0.4,0.7,0.8]})

cust_df = cust_df.copy().loc[cust_df.index.repeat(n_repeats)].reset_index(drop=True)
cust_df['account_number'] = random.sample(range(100, 10000000), len(cust_df))
if (cust_df.account_number.value_counts()>1).sum() > 0:
    raise ValueError('Dupe account number exists!')
print('{} Customers Considered'.format(len(cust_df)))
#### Available Offers i.e. Decision Variables ####
dec_variables = {
    'sports_offers' : [0,3,7,10,13],
    'cine_offers': [0,2,4,6,10],
    'kids_offers': [0,1,2,4,5],
    'sig_offers': [0,1,2,4,5],
    'hd_offers': [0,1,2]
                }
decision_variables = {'sports_offers':['sports_0','sports_3','sports_7','sports_10','sports_13'],
                      'kids_offers':['kids_0','kids_1','kids_2','kids_4','kids_5'],
                      'cine_offers':['cine_0','cine_2','cine_4','cine_6','cine_10'],
                     'sig_offers':['sig_0','sig_1','sig_2','sig_4','sig_5'],
                     'hd_offers':['hd_0','hd_1','hd_2']}

net_contribution = {'sports':14,'kids':4,'cine':8,'sig':6,'hd':3}
# Some number to use as a power to prevent NC dominating the formula (if huge difference in NC between products)
net_contribution_scaler = 0.2

num_accts = cust_df.shape[1]
num_offers = len([i for i in cust_df.columns if 'Active' in i])

4 Customers Considered


### Solution 1 - Mixed Integer (MIP) Programming Approach

- Cannot seem to find a way to define a domain of integer values to choose from, only lb and ub approach
- Need to use binary defined variables 
- Objective function is key
- Cannot use GLOP as includes floating point solutions 
- 'SCIP' backend handles mixed integer but a lot slower than pure LP
- Solution is singular
    - Can we iterate through and add constraint that cannot be previous solution so have multiple "optimal" solutions OR is this the point of CP??
    
- __Allocation Score Formula__
    - At a product level:
        - Product_Score = NetContribution^power_scaler * DG_Proba * OfferAmt (£)
        - FinalScore = SUM(Product_Score) for all products
    - Prioritises giving deeper offers to products with higher NC and higher DG propensity
    - N.B. Big assumption that DG probability is an accurate measure to use here
    - Alternatives?
        - Expected NC e.g. NC*(1-DG_Proba) but would likely want to calibrate this DG_proba

In [15]:
detailed_outputs = True
# Store results #
acct_dict = {}
bto_dict = {}
total_time_ms = 0

for k in decision_variables.keys():
    all_offers = decision_variables.values()
    flat_list = []
    for sublist in all_offers:
        for item in sublist:
            flat_list.append(item)        

offer_vector = [int(re.findall(r'\d+', s)[0]) for s in flat_list]
sports_offer_vector = [int(re.findall(r'\d+', i)[0]) for i in flat_list if 'sports' in i]
kids_offer_vector = [int(re.findall(r'\d+', i)[0]) for i in flat_list if 'kids' in i]
cine_offer_vector = [int(re.findall(r'\d+', i)[0]) for i in flat_list if 'cine' in i]
hd_offer_vector = [int(re.findall(r'\d+', i)[0]) for i in flat_list if 'hd' in i]
sig_offer_vector = [int(re.findall(r'\d+', i)[0]) for i in flat_list if 'sig' in i]

sports_max_score = 1*net_contribution['sports']*max(sports_offer_vector)
kids_max_score = 1*net_contribution['kids']*max(kids_offer_vector)
cine_max_score = 1*net_contribution['cine']*max(cine_offer_vector)
hd_max_score = 1*net_contribution['hd']*max(hd_offer_vector)
sig_max_score = 1*net_contribution['sig']*max(sig_offer_vector)

start = time.time()
for acct in cust_df.account_number:
    acct_dict[acct] = []
    bto_dict[acct] = []
    
    # Instantiate solver #
    solver = pywraplp.Solver.CreateSolver('SCIP') # SCIP/CBC/ - CANNOT SIMPLY CHANGE THESE E.G. CPLEX DOESNT ALLOW INTVAR, RAISES ERROR 
    
    ls_cap = int(cust_df.loc[cust_df['account_number']==acct,'LS_Cap'])
    bto_low = int(np.where(cust_df.loc[cust_df['account_number']==acct,'BTO_low']>=ls_cap,ls_cap-2,cust_df.loc[cust_df['account_number']==acct,'BTO_low'].values[0])) # arbitrarily set to £2 less than LS cap
    bto_high = int(np.where(cust_df.loc[cust_df['account_number']==acct,'BTO_high']>=ls_cap,ls_cap,cust_df.loc[cust_df['account_number']==acct,'BTO_high'].values[0]))
    bto_target = int(cust_df.loc[cust_df['account_number']==acct,'BTO_target'])
    sports_active = int(cust_df.loc[cust_df['account_number'] == acct,'Sports_Active'])
    kids_active = int(cust_df.loc[cust_df['account_number'] == acct,'Kids_Active'])
    cine_active = int(cust_df.loc[cust_df['account_number'] == acct,'Cine_Active'])
    hd_active = int(cust_df.loc[cust_df['account_number'] == acct,'HD_Active'])
    sports_dg = float(cust_df.loc[cust_df['account_number'] == acct,'Sports_DG'])
    cine_dg = float(cust_df.loc[cust_df['account_number'] == acct,'Cine_DG'])
    kids_dg = float(cust_df.loc[cust_df['account_number'] == acct,'Kids_DG']) 
    hd_dg = float(cust_df.loc[cust_df['account_number'] == acct,'HD_DG']) 
    sig_dg = float(cust_df.loc[cust_df['account_number'] == acct,'Signature_DG']) 
    
    # Define decision variables #
    variable_dict = {k:[] for k,v in decision_variables.items()}
    x = {}
    for k in decision_variables.keys():
        for offer in decision_variables[k]:
            variable_dict[k].append(solver.IntVar(0, 1, offer)) # equally valid is solver.BoolVar(offer)
            
    ### Constraints applied ###
    # 1. Do not give offer > 0 where product inactive # 
    if sports_active!=1:
        solver.Add(variable_dict['sports_offers'][0] == 1)
    if kids_active!=1:
        solver.Add(variable_dict['kids_offers'][0] == 1)
    if cine_active!=1:
        solver.Add(variable_dict['cine_offers'][0] == 1)
    if hd_active!=1:
        solver.Add(variable_dict['hd_offers'][0] == 1)
    
    # 2. Only one offer per product type #  
    for k in variable_dict.keys():
        solver.Add(sum(variable_dict[k]) == 1)
    
    # 3. Sum of total offers must be less than the LS Cap - Need to multiply by vector of actual offers #
    flat_integers = []
    for sublist in variable_dict.values():
        for item in sublist:
            flat_integers.append(item)
    # solver.Add( sum({k:sum(v) for k,v in variable_dict.items()}.values()) <= ls_cap)
    solver.Add( sum(np.array(offer_vector)*np.array(flat_integers)) <= ls_cap )

    ##### Objective function that relies on absolute values hard to define - as abs() cannot be used in linear formulation - so cannot use diff with LP - AddMaxEquality does not exist for LP problems #####
    # Create auxilliary variable : https://math.stackexchange.com/questions/2446606/linear-programming-set-a-variable-the-max-between-two-another-variables/2447498#2447498 #
    # Want the max(A,B) i.e. max(pos_delta,neg_delta) which is the the absolute miss value. Overall optimisation aim: min(max(pos_delta,neg_delta))
    
    dist_variable = solver.IntVar(-bto_target,bto_target,'aux_variable')
    
    # Add constraints to this auxilliary variable #
    solver.Add( dist_variable >= (bto_target - sum(np.array(offer_vector)*np.array(flat_integers))) )
    solver.Add( dist_variable >= ((sum(np.array(offer_vector)*np.array(flat_integers)) - bto_target)) )
    
    sports_allocation_score = sum(sports_dg * (np.array(sports_offer_vector)*np.array(variable_dict['sports_offers'])) * net_contribution['sports']**net_contribution_scaler)
    kids_allocation_score = sum(kids_dg * (np.array(kids_offer_vector)*np.array(variable_dict['kids_offers'])) * net_contribution['kids']**net_contribution_scaler)
    cine_allocation_score = sum(cine_dg * (np.array(cine_offer_vector)*np.array(variable_dict['cine_offers'])) * net_contribution['cine']**net_contribution_scaler)
    hd_allocation_score = sum(hd_dg * (np.array(hd_offer_vector)*np.array(variable_dict['hd_offers'])) * net_contribution['hd']**net_contribution_scaler)
    sig_allocation_score = sum(sig_dg * (np.array(sig_offer_vector)*np.array(variable_dict['sig_offers'])) * net_contribution['sig']**net_contribution_scaler)
    
    # MinMax scaler 0-1 #
    # The larger this is, the better the allocation is i.e. the more distributed offers are towards highDG propensity & high NC products # 
    allocation_score_minmax = (((sports_allocation_score + kids_allocation_score + cine_allocation_score + hd_allocation_score + sig_allocation_score) - 0) /
                               (sports_max_score+kids_max_score+cine_max_score+hd_max_score+sig_max_score))
    
    # Optimise #
    solver.Minimize(dist_variable+(1-allocation_score_minmax))
    
    # Store Results #
    status = solver.Solve()
    if status == solver.OPTIMAL:
        bto_dict[acct] += [np.floor(solver.Objective().Value())] # dont think this is working atm 
        solutions_list = []
        for k in variable_dict.keys():
            for offer in variable_dict[k]:
                # print(offer,offer.solution_value())
                solutions_list+=[offer.solution_value()]
        acct_dict[acct].append(solutions_list)

        if detailed_outputs:
            print(acct)
            print(acct_dict[acct])
            print('BTO TARGET ',bto_target)
            print('BTO Target Miss (£) =', np.floor(solver.Objective().Value()))
            print('BTO Miss including Allocation (£) =', solver.Objective().Value())
            print('Solved using {} branch and bound nodes'.format(solver.nodes()))
            print('Solved using {} iterations'.format(solver.iterations()))
    else:
        if detailed_outputs:
            print('No Solution Found! Status {}'.format(status))
        pass
    
    total_time_ms += solver.wall_time()

print(acct_dict)
print(bto_dict)
print('Time taken to find all solutions: {} mins'.format(total_time_ms/ 60000))

6999
[[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]]
BTO TARGET  12
BTO Target Miss (£) = 0.0
BTO Miss including Allocation (£) = 0.9487734734456031
Solved using 1 branch and bound nodes
Solved using 17 iterations
8003604
[[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.0, 0.0, 1.0, 0.0, -0.0, 1.0, 0.0, 0.0]]
BTO TARGET  16
BTO Target Miss (£) = 1.0
BTO Miss including Allocation (£) = 1.951871793582848
Solved using 1 branch and bound nodes
Solved using 13 iterations
2915992
[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.0]]
BTO TARGET  6
BTO Target Miss (£) = 0.0
BTO Miss including Allocation (£) = 0.9778020646410944
Solved using 1 branch and bound nodes
Solved using 9 iterations
6196557
[[-0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, -0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]]
BTO TARGET  3

# Solution 3

__Constraint Programming Approach (need to do allocation external to optimisation afterwards)__
- Solution is a subset of feasible solutions

In [8]:
# Store feasible offer combinations #
use_objective = False
acct_dict = {}

for acct in cust_df.account_number:
    print(acct)
    model = cp_model.CpModel()
    
    ls_cap = cust_df.loc[cust_df['account_number']==acct,'LS_Cap'].values[0]
    bto_low = int(np.where(cust_df.loc[cust_df['account_number']==acct,'BTO_low']>=ls_cap,ls_cap-2,cust_df.loc[cust_df['account_number']==acct,'BTO_low'].values[0])) # arbitrarily set to £2 less than LS cap
    bto_high = int(np.where(cust_df.loc[cust_df['account_number']==acct,'BTO_high']>=ls_cap,ls_cap,cust_df.loc[cust_df['account_number']==acct,'BTO_high'].values[0]))
    bto_target = int(cust_df.loc[cust_df['account_number']==acct,'BTO_target'])
    sports_holding = int(np.where(cust_df.loc[cust_df['account_number']==acct,'Sports_Active']==1,int(np.array(dec_variables['sports_offers']).max()),0))
    cine_holding = int(np.where(cust_df.loc[cust_df['account_number']==acct,'Cine_Active']==1,int(np.array(dec_variables['cine_offers']).max()),0))
    kids_holding = int(np.where(cust_df.loc[cust_df['account_number']==acct,'Kids_Active']==1,int(np.array(dec_variables['kids_offers']).max()),0))
    
    sports = model.NewIntVarFromDomain(cp_model.
                          # Domain.FromIntervals([dec_variables['sports_offers']]), name='sports'
                                       Domain.FromIntervals([[i] for i in dec_variables['sports_offers']]), name='sports'
                                      )
    kids = model.NewIntVarFromDomain(cp_model.
                          # Domain.FromIntervals([dec_variables['kids_offers']]), name='kids'
                                     Domain.FromIntervals([[i] for i in dec_variables['kids_offers']]), name='kids'
                                    )
    cine = model.NewIntVarFromDomain(cp_model.
                                     # Domain.FromIntervals([dec_variables['cine_offers']]), name='cine'
                                     Domain.FromIntervals([[i] for i in dec_variables['cine_offers']]), name='cine'
                                    )
    
    ##### Optional objective function #####
    if use_objective:
        delta_1 = (bto_target-(sports+kids+cine))
        delta_2 = ((sports+kids+cine) - bto_target)
        delta = model.NewIntVar(0,100, 'delta')
        model.AddMaxEquality(delta,[delta_1,delta_2]) # needs array of variables, not array of constraints here
        # reason is that calling abs() on a linear expression is not supported, ''please use CpModel.AddAbsEquality'e.g. cannot abs_delta = (max(delta_1, delta_2)) # gives the absolute value of this 
        model.Minimize(delta)
    
    ##### Constraints #######
    
    # Sum of total offers must be less than the LS Cap #
    model.Add(sports+kids+cine < ls_cap)
    model.Add(sports_holding >= sports)
    model.Add(cine_holding >= cine)
    model.Add(kids_holding >= kids)
    
    # Sum of offers must be between BTO range 
    model.Add( bto_low <= sports+kids+cine)
    model.Add( bto_high >= sports+kids+cine)
    
    # Need to check solver status before accessing the solution # 
    # ###### APPROACH 1 - WORKS WITH OR WITHOUT OBJECTIVE FUNCTION #######
    if use_objective:
        solution_printer = VarArraySolutionPrinter([sports, kids, cine])
        solver = cp_model.CpSolver()
        # Enumerate all solutions # 
        solver.parameters.enumerate_all_solutions = True
        status = solver.Solve(model, solution_printer)
        acct_dict[acct] = solution_printer.solution_list()
        # print('Status = %s' % solver.StatusName(status))
        # print('Number of solutions found: %i' % solution_printer.solution_count())
        print('Objective Value=',solver.ObjectiveValue())
    else:
        solution_callback = cp_model.CpSolverSolutionCallback()
        solver = cp_model.CpSolver()
        # Enumerate all solutions # 
        solver.parameters.enumerate_all_solutions = True
            
        solution_collector = VarArraySolutionCollector([sports, kids, cine])
        solver.SearchForAllSolutions(model, solution_collector)
        # print('Status = %s' % solver.StatusName(status))
        # print('Number of solutions found: %i' % solution_printer.solution_count())
        # print('Solutions List',solution_collector.solution_list)
        unique_solutions = [list(x) for x in set(tuple(x) for x in solution_collector.solution_list)]
        print('Unique Solutions List',unique_solutions)
        acct_dict[acct] = unique_solutions

6999
Unique Solutions List [[3, 4, 6], [3, 5, 4], [7, 5, 0], [7, 2, 2], [0, 5, 6], [7, 1, 2], [10, 0, 0], [3, 1, 6], [10, 0, 2], [7, 0, 4], [7, 4, 0], [10, 1, 2], [13, 0, 0], [7, 0, 6], [3, 0, 10], [7, 4, 2], [7, 1, 4], [10, 1, 0], [10, 2, 0], [3, 2, 6], [0, 1, 10], [3, 5, 2], [0, 0, 10], [3, 4, 4], [0, 2, 10], [7, 2, 4], [0, 4, 6]]
8003604
Unique Solutions List [[3, 4, 6], [7, 2, 4], [3, 1, 10], [7, 0, 6], [0, 4, 10], [7, 1, 6], [10, 1, 2], [10, 2, 2], [13, 1, 0], [7, 4, 2], [13, 0, 0], [3, 5, 6], [7, 5, 2], [10, 4, 0], [3, 0, 10], [10, 0, 4]]
2915992
Unique Solutions List [[0, 5, 0]]
6196557
Unique Solutions List []


In [ ]:
# def allocation_algorithm(offers:list,
#                          contributions:list,
#                          power:float=0.1,
#                          dg_probas:list):
#     """
#     params:
#         dg_probas: could either be fixed for each customer based on respective DG propensity model OR updated wrt specific product offer_amt (relies on building separate product models)
#         power: transforms the net contributions of each product to reduce how dominant it is in the allocation formula (hyperparameter to test and learn)
    
#     return:
#         optimal_offers_list e.g. [10,2,4]
#     """
#     contribution_importance = [pow(i,power) for i in contributions]
#     offers * dg_probas * contributions
    
# # If no solutions - why could this be? Not enough products & deep enough offers to get within BTO range? Maybe just take the max discount for all the products
# if len() == 1:
#     pass
    
# if len() == 0:
    
# # Allocation algorithm #
# if len() > 1: